In [1]:
import requests 
from tqdm.auto import tqdm
from elasticsearch import Elasticsearch


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
es_client = Elasticsearch(
    "http://localhost:9200")


In [3]:
es_client.info()

ObjectApiResponse({'name': 'dff54edc8ed7', 'cluster_name': 'docker-cluster', 'cluster_uuid': '96a64p6oTvmiO-D3GdPbiA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
index_settings =  {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}


index_name='homework'


In [6]:

# es_client.indices.create(
#     index=index_name,
#     body=index_settings
# )

In [7]:
for doc in tqdm(documents):
    es_client.index(
        index=index_name,
        document=doc
    )

100%|██████████| 948/948 [00:02<00:00, 394.06it/s]


In [8]:
query3 = 'How do execute a command on a Kubernetes pod?'
search_query_3 ={
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query3,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },

        }
    }
    }

In [9]:
def elastic_search(search_query):
    respone = es_client.search(index=index_name, body=search_query)
    results_docs = []
    results_score = []
    for hit in respone['hits']['hits']:
        results_docs.append(hit['_source'])
        results_score.append(hit['_score'])
    return results_docs,results_score

In [10]:
_,scores = elastic_search(search_query_3)

In [11]:
scores


[44.56891]

In [12]:
query4 = 'How do copy a file to a Docker container?'
search_query_4 ={
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query4,
                    "fields": ["question", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
    }

In [13]:
search_results_4,_ = elastic_search(search_query_4)

In [14]:
search_results_4[2]

{'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
 'section': '5. Deploying Machine Learning Models',
 'question': 'How do I copy files from my local machine to docker container?',
 'course': 'machine-learning-zoomcamp'}

In [15]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context="" 

    for doc in search_results:
        context += f"Q: {doc['question']}\n"
        context += f"A: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [16]:
q5 =  "How do copy a file to a Docker container?"

prompt = build_prompt(q5, search_results_4)


In [17]:
len(prompt)

1537

In [18]:
import tiktoken

In [19]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [20]:
len(encoding.encode(prompt))

334